# 🧭 Sistema de Router Chains - Guia de Viagem

**Objetivo**: Implementar um sistema inteligente que classifica consultas turísticas e direciona para chains especializadas.

## 🎯 Tipos de Consulta Suportadas:
1. **`roteiro-viagem`**: Planejamento de roteiros e itinerários
2. **`logistica-transporte`**: Informações sobre transporte e logística
3. **`info-local`**: Dados sobre pontos turísticos, restaurantes, cultura
4. **`traducao-idiomas`**: Tradução e comunicação em outros idiomas

## 📋 Arquitetura:
- **Router Chain**: Classifica a intenção da consulta
- **Chains Especializadas**: Processam cada tipo de consulta
- **RAG Integration**: Usa dados do Pinecone para respostas contextualizadas

In [41]:
# Imports e configuração inicial
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Carregar variáveis de ambiente
load_dotenv()

print("✅ Imports carregados com sucesso!")

✅ Imports carregados com sucesso!


In [42]:
# Inicializar componentes principais
print("🔄 Inicializando componentes...")

# LLM Groq - usando modelo atualizado
llm = ChatGroq(
    temperature=0.1,
    model="llama-3.1-8b-instant",  # Modelo atualizado e suportado
    groq_api_key=os.getenv('GROQ_API_KEY')
)

# Pinecone e embeddings (do notebook anterior)
pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
indice = pinecone_client.Index('guia-viagem')
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("✅ Componentes inicializados:")
print(f"🤖 LLM: {llm.model_name}")
print(f"🗂️ Pinecone: {indice.describe_index_stats()['total_vector_count']} vetores")
print(f"🎯 Embeddings: {embeddings_model.model_name}")

🔄 Inicializando componentes...
✅ Componentes inicializados:
🤖 LLM: llama-3.1-8b-instant
✅ Componentes inicializados:
🤖 LLM: llama-3.1-8b-instant
🗂️ Pinecone: 4 vetores
🎯 Embeddings: sentence-transformers/all-MiniLM-L6-v2
🗂️ Pinecone: 4 vetores
🎯 Embeddings: sentence-transformers/all-MiniLM-L6-v2


In [43]:
# 🗺️ TEMPLATE 1: ROTEIRO DE VIAGEM
roteiro_template = """
Você é um especialista em planejamento de roteiros turísticos. Use o contexto fornecido para criar roteiros detalhados.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Crie um roteiro estruturado e detalhado
- Inclua horários sugeridos e duração das atividades
- Considere proximidade geográfica dos pontos
- Adicione dicas práticas e recomendações

RESPOSTA:
"""

print("✅ Template ROTEIRO DE VIAGEM definido!")
print(f"📏 Tamanho: {len(roteiro_template)} caracteres")

✅ Template ROTEIRO DE VIAGEM definido!
📏 Tamanho: 389 caracteres


In [44]:
# 🚗 TEMPLATE 2: LOGÍSTICA E TRANSPORTE
logistica_template = """
Você é um especialista em logística de viagem e sistemas de transporte urbano. Use o contexto para orientar sobre mobilidade.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Forneça informações detalhadas sobre transporte
- Inclua custos, horários e rotas quando possível
- Sugira alternativas de transporte
- Dê dicas de segurança e eficiência

RESPOSTA:
"""

print("✅ Template LOGÍSTICA E TRANSPORTE definido!")
print(f"📏 Tamanho: {len(logistica_template)} caracteres")

✅ Template LOGÍSTICA E TRANSPORTE definido!
📏 Tamanho: 385 caracteres


In [45]:
# 🏛️ TEMPLATE 3: INFORMAÇÕES LOCAIS
info_local_template = """
Você é um guia turístico local expert. Use o contexto fornecido para dar informações detalhadas sobre pontos turísticos, cultura e recomendações.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Forneça informações detalhadas e precisas
- Inclua dados históricos, culturais ou curiosidades
- Adicione recomendações práticas (horários, preços, dicas)
- Mantenha um tom acolhedor e informativo

RESPOSTA:
"""

print("✅ Template INFORMAÇÕES LOCAIS definido!")
print(f"📏 Tamanho: {len(info_local_template)} caracteres")

✅ Template INFORMAÇÕES LOCAIS definido!
📏 Tamanho: 431 caracteres


In [46]:
# 🗣️ TEMPLATE 4: TRADUÇÃO E IDIOMAS
traducao_template = """
Você é um assistente de tradução e comunicação intercultural. Ajude com traduções, frases úteis e orientações culturais.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Forneça traduções precisas e contextualmente adequadas
- Inclua frases úteis para situações comuns de viagem
- Dê dicas sobre costumes culturais de comunicação
- Adicione informações sobre pronúncia quando relevante

RESPOSTA:
"""

print("✅ Template TRADUÇÃO E IDIOMAS definido!")
print(f"📏 Tamanho: {len(traducao_template)} caracteres")

✅ Template TRADUÇÃO E IDIOMAS definido!
📏 Tamanho: 425 caracteres


In [47]:
# Função para buscar contexto relevante no Pinecone
def buscar_contexto_relevante(consulta, top_k=3):
    """
    Busca informações relevantes no Pinecone baseado na consulta do usuário.
    """
    try:
        # Gerar embedding da consulta
        query_embedding = embeddings_model.embed_documents([consulta])[0]
        
        # Buscar no Pinecone
        resultado = indice.query(
            vector=query_embedding, 
            top_k=top_k, 
            include_metadata=True
        )
        
        # Extrair textos relevantes
        contextos = []
        for match in resultado['matches']:
            if match['score'] > 0.3:  # Filtro de relevância
                contextos.append(match['metadata']['text'])
        
        return "\n\n".join(contextos) if contextos else "Nenhum contexto específico encontrado na base de conhecimento."
    
    except Exception as e:
        print(f"Erro na busca: {e}")
        return "Erro ao buscar contexto na base de conhecimento."

# Testar a função
teste_contexto = buscar_contexto_relevante("pontos turísticos do Rio de Janeiro")
print("🔍 Teste da busca de contexto:")
print(f"Primeiros 200 caracteres: {teste_contexto[:200]}...")

🔍 Teste da busca de contexto:
Primeiros 200 caracteres: # Base de Conhecimento - Rio de Janeiro
Pão de Açúcar: Um dos pontos turísticos mais famosos do Rio, com vista panorâmica da cidade.
Cristo Redentor: Uma das sete maravilhas do mundo moderno, localiza...


In [48]:
# Criar chains especializadas com RAG
def criar_chain_com_rag(template, nome):
    """
    Cria uma chain que integra RAG (busca no Pinecone) com o template específico.
    """
    prompt = PromptTemplate(
        template=template,
        input_variables=["input", "context"]
    )
    
    def chain_com_contexto(consulta):
        # Buscar contexto relevante
        contexto = buscar_contexto_relevante(consulta)
        
        # Executar chain com contexto
        chain = LLMChain(llm=llm, prompt=prompt)
        return chain.invoke({"input": consulta, "context": contexto})["text"]
    
    return chain_com_contexto

# Criar as 4 chains especializadas
chains = {
    "roteiro-viagem": criar_chain_com_rag(roteiro_template, "Roteiro de Viagem"),
    "logistica-transporte": criar_chain_com_rag(logistica_template, "Logística e Transporte"),
    "info-local": criar_chain_com_rag(info_local_template, "Informações Locais"),
    "traducao-idiomas": criar_chain_com_rag(traducao_template, "Tradução e Idiomas")
}

print("✅ Chains especializadas criadas:")
for nome in chains.keys():
    print(f"📋 {nome}")

✅ Chains especializadas criadas:
📋 roteiro-viagem
📋 logistica-transporte
📋 info-local
📋 traducao-idiomas


In [49]:
# Criar o Router Chain para classificar consultas
router_template = """
Você é um classificador de consultas turísticas. Analise a consulta do usuário e determine qual categoria ela pertence.

CATEGORIAS DISPONÍVEIS:
- roteiro-viagem: Planejamento de roteiros, itinerários, programação de atividades
- logistica-transporte: Transporte, como chegar, mobilidade urbana, horários
- info-local: Informações sobre pontos turísticos, restaurantes, cultura, história
- traducao-idiomas: Tradução, frases úteis, comunicação, idiomas

CONSULTA: {input}

INSTRUÇÕES:
- Responda APENAS com o nome da categoria
- Use exatamente os nomes: roteiro-viagem, logistica-transporte, info-local, ou traducao-idiomas
- Se não tiver certeza, use 'info-local' como padrão

CATEGORIA:
"""

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"]
)

router_chain = LLMChain(llm=llm, prompt=router_prompt)

print("✅ Router Chain criado para classificação de consultas!")

✅ Router Chain criado para classificação de consultas!


In [50]:
# Sistema principal - Guia de Viagem Inteligente
def guia_viagem_inteligente(consulta_usuario):
    """
    Sistema principal que processa consultas turísticas:
    1. Classifica a consulta
    2. Direciona para a chain apropriada
    3. Retorna resposta contextualizada com RAG
    """
    print(f"🔍 Processando: '{consulta_usuario}'")
    
    try:
        # Passo 1: Classificar a consulta
        categoria = router_chain.invoke({"input": consulta_usuario})["text"].strip().lower()
        print(f"📂 Categoria: {categoria}")
        
        # Passo 2: Verificar se a categoria é válida
        if categoria not in chains:
            print(f"⚠️ Categoria '{categoria}' não reconhecida. Usando 'info-local'.")
            categoria = "info-local"
        
        # Passo 3: Executar a chain especializada
        resposta = chains[categoria](consulta_usuario)
        
        # Passo 4: Retornar resultado estruturado
        resultado = {
            "consulta": consulta_usuario,
            "categoria": categoria,
            "resposta": resposta
        }
        
        return resultado
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        return {
            "consulta": consulta_usuario,
            "categoria": "erro",
            "resposta": f"Desculpe, ocorreu um erro ao processar sua consulta: {e}"
        }

print("✅ Sistema Guia de Viagem Inteligente configurado!")
print("🎯 Pronto para processar consultas turísticas!")

✅ Sistema Guia de Viagem Inteligente configurado!
🎯 Pronto para processar consultas turísticas!


In [51]:
# TESTE 1: Info Local - Pontos Turísticos
print("🧪 TESTE 1: Informações Locais")
print("="*50)

consulta = "Quais são os principais pontos turísticos do Rio de Janeiro?"
resultado = guia_viagem_inteligente(consulta)

print(f"\n📝 RESULTADO:")
print(f"📂 Categoria: {resultado['categoria']}")
print(f"💬 Resposta completa:")
print(resultado['resposta'])

🧪 TESTE 1: Informações Locais
🔍 Processando: 'Quais são os principais pontos turísticos do Rio de Janeiro?'


📂 Categoria: info-local

📝 RESULTADO:
📂 Categoria: info-local
💬 Resposta completa:
Olá, bem-vindo ao Rio de Janeiro! É um prazer ter você aqui. Nossa cidade é conhecida por sua beleza natural, rica cultura e pontos turísticos incríveis. Vou compartilhar com você os principais pontos turísticos do Rio, juntamente com algumas dicas práticas e curiosidades interessantes.

**1. Pão de Açúcar**

O Pão de Açúcar é um dos pontos turísticos mais famosos do Rio, e por boa razão. Localizado na Ilha de Paquetá, oferece uma vista panorâmica deslumbrante da cidade. Você pode subir ao topo do morro de forma segura, utilizando um teleférico que leva apenas 2 minutos para chegar ao topo. A vista é simplesmente incrível, especialmente durante o pôr do sol.

- Horário de funcionamento: 8h às 20h
- Preço: R$ 62,00 (ida e volta)
- Dica: Evite visitar durante a temporada de chuva, pois o teleférico pode ser interrompido.

**2. Cristo Redentor**

O Cristo Redentor é uma das sete maravilhas do mundo moderno 

In [52]:
# TESTE 2: Roteiro de Viagem
print("🗓️ TESTE 2: Planejamento de Roteiro")
print("="*50)

consulta = "Como posso criar um roteiro de 2 dias em Paris?"
resultado = guia_viagem_inteligente(consulta)

print(f"\n📝 RESULTADO:")
print(f"📂 Categoria: {resultado['categoria']}")
print(f"💬 Resposta completa:")
print(resultado['resposta'])

🗓️ TESTE 2: Planejamento de Roteiro
🔍 Processando: 'Como posso criar um roteiro de 2 dias em Paris?'
📂 Categoria: roteiro-viagem
📂 Categoria: roteiro-viagem

📝 RESULTADO:
📂 Categoria: roteiro-viagem
💬 Resposta completa:
**Roteiro de 2 Dias em Paris**

**Dia 1:**

**Manhã (9h00 - 13h00)**

* 9h00: Inicie o dia visitando a **Torre Eiffel**. Compre ingressos com antecedência para evitar filas. A torre está aberta das 9h30 às 23h45.
* 11h00: Após visitar a Torre Eiffel, dirija-se ao **Champ de Mars**, um parque ao lado da torre com uma vista incrível da cidade.
* 12h30: Almoce em um dos restaurantes da região, como o **Le Relais de l'Entrecôte**, conhecido por sua comida francesa clássica.

**Tarde (14h00 - 18h00)**

* 14h30: Visite o **Museu do Louvre**, um dos maiores museus de arte do mundo. Compre ingressos com antecedência para evitar filas. O museu está aberto das 9h00 às 18h00.
* 17h00: Após visitar o museu, dirija-se ao **Jardim das Tulherias**, um parque ao lado do museu com uma v

In [53]:
# TESTE 3: Logística e Transporte
print("🚌 TESTE 3: Informações de Transporte")
print("="*50)

consulta = "Como chegar do aeroporto ao centro do Rio?"
resultado = guia_viagem_inteligente(consulta)

print(f"\n📝 RESULTADO:")
print(f"📂 Categoria: {resultado['categoria']}")
print(f"💬 Resposta completa:")
print(resultado['resposta'])

🚌 TESTE 3: Informações de Transporte
🔍 Processando: 'Como chegar do aeroporto ao centro do Rio?'
📂 Categoria: logistica-transporte
📂 Categoria: logistica-transporte

📝 RESULTADO:
📂 Categoria: logistica-transporte
💬 Resposta completa:
Chegar ao centro do Rio de Janeiro a partir do aeroporto pode ser um desafio, mas com as informações certas, você pode planejar sua viagem de forma eficiente e segura. Aqui estão as opções de transporte disponíveis:

**1. Táxi**

* Custo: R$ 60 a R$ 80 (aproximadamente 12 a 16 euros)
* Horário: 24 horas
* Rota: O aeroporto Internacional do Rio de Janeiro (Galeão) está localizado a cerca de 20 km do centro da cidade. O táxi leva em torno de 30 a 40 minutos, dependendo do tráfego.

**2. Ônibus**

* Custo: R$ 5 a R$ 10 (aproximadamente 1 a 2 euros)
* Horário: 24 horas
* Rota: O aeroporto tem uma parada de ônibus, onde você pode pegar o ônibus 2018 ou 2145, que levam ao centro da cidade. A viagem leva em torno de 45 a 60 minutos.

**3. Metrô e Ônibus**

* Cust

In [54]:
# TESTE 4: Tradução e Idiomas
print("🌍 TESTE 4: Tradução e Comunicação")
print("="*50)

consulta = "Como dizer 'onde fica o banheiro?' em francês?"
resultado = guia_viagem_inteligente(consulta)

print(f"\n📝 RESULTADO:")
print(f"📂 Categoria: {resultado['categoria']}")
print(f"💬 Resposta completa:")
print(resultado['resposta'])

🌍 TESTE 4: Tradução e Comunicação
🔍 Processando: 'Como dizer 'onde fica o banheiro?' em francês?'
📂 Categoria: traducao-idiomas
📂 Categoria: traducao-idiomas

📝 RESULTADO:
📂 Categoria: traducao-idiomas
💬 Resposta completa:
Uma pergunta comum para viajantes!

A tradução de "onde fica o banheiro?" em francês é:

"Où est les toilettes?"

Se você quiser ser mais formal, pode dizer:

"Excusez-moi, où sont les toilettes, s'il vous plaît?"

Aqui estão algumas dicas adicionais sobre comunicação em francês:

- Pronúncia: A palavra "toilettes" é pronunciada como "twalét" (com um som de "tw" suave).
- Costumes culturais: Nos restaurantes e cafés, é comum que os banheiros sejam localizados em um canto ou em um andar inferior. Não hesite em perguntar se você não encontrar.
- Frases úteis:
 + "Je voudrais une carte, s'il vous plaît" (Eu gostaria de um mapa, por favor)
 + "Combien ça coûte?" (Quanto custa?)
 + "Je suis perdu(e), pouvez-vous m'aider?" (Eu estou perdido, pode me ajudar?)

Lembre-se de 

In [55]:
# Teste personalizado - Faça sua própria consulta!
print("🎯 Teste seu próprio caso:")
print("Descomente e modifique a linha abaixo com sua consulta:")
print()

# Exemplo de consulta personalizada:

# minha_consulta = "Onde posso comer bem no Rio de Janeiro?"
# resultado_personalizado = guia_viagem_inteligente(minha_consulta)
# print(f"Categoria: {resultado_personalizado['categoria']}")
# print(f"Resposta: {resultado_personalizado['resposta']}")

print("="*60)
print("🎉 SISTEMA GUIA DE VIAGEM INTELIGENTE CONCLUÍDO!")
print("="*60)
print("✅ Router Chain: Classificação automática funcionando")
print("✅ RAG Integration: Pinecone + Embeddings operacional") 
print("✅ 4 Chains Especializadas: Roteiro, Logística, Info Local, Tradução")
print("✅ Groq LLM: Modelo llama-3.1-8b-instant funcionando")
print("✅ Base de Conhecimento: 4 chunks turísticos indexados")
print()
print("🚀 Seu assistente turístico inteligente está pronto para uso!")
print("📋 Execute consultas personalizadas descomentando o código acima.")

🎯 Teste seu próprio caso:
Descomente e modifique a linha abaixo com sua consulta:

🎉 SISTEMA GUIA DE VIAGEM INTELIGENTE CONCLUÍDO!
✅ Router Chain: Classificação automática funcionando
✅ RAG Integration: Pinecone + Embeddings operacional
✅ 4 Chains Especializadas: Roteiro, Logística, Info Local, Tradução
✅ Groq LLM: Modelo llama-3.1-8b-instant funcionando
✅ Base de Conhecimento: 4 chunks turísticos indexados

🚀 Seu assistente turístico inteligente está pronto para uso!
📋 Execute consultas personalizadas descomentando o código acima.
